# RBE/CS549 Fall 2022: Computer Vision
## Homework 0: Alohomora

Author(s): 
Prof. Nitin J. Sanket (nsanket@wpi.edu), Lening Li (lli4@wpi.edu), Gejji, Vaishnavi Vivek (vgejji@wpi.edu)

Robotics Engineering Department,

Worcester Polytechnic Institute

Code adapted from CMSC733 at the University of Maryland, College Park.

## Phase 2

### Neural Network Construction

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def loss_fn(out, labels):
    ###############################################
    # Fill your loss function of choice here!
    ###############################################
    loss = nn.CrossEntropyLoss()
    loss = loss(out, labels)
    return loss

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = loss_fn(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = loss_fn(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'loss': loss.detach(), 'acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'loss': epoch_loss.item(), 'acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], loss: {:.4f}, acc: {:.4f}".format(epoch, result['loss'], result['acc']))



class CIFAR10Model(ImageClassificationBase):
    def __init__(self, InputSize, OutputSize):
        """
        Inputs: 
        InputSize - Size of the Input
        OutputSize - Size of the Output
        """
        #############################
        # Fill your network initialization of choice here!
        #############################
        super().__init__()
        self.linear1 = nn.Linear(InputSize, 1024)
        self.linear2 = nn.Linear(1024, 512)
        self.linear3 = nn.Linear(512, 64)
        self.linear4 = nn.Linear(64, 64)
        self.linear5 = nn.Linear(64, OutputSize)

    def forward(self, xb):
        """
        Input:
        xb is a MiniBatch of the current image
        Outputs:
        out - output of the network
        """
        #############################
        # Fill your network structure of choice here!
        #############################
        out = xb.view(xb.size(0), -1)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        out = F.relu(out)
        out = self.linear4(out)
        out = F.relu(out)
        out = self.linear5(out)
        return out


In [13]:
import time
import glob
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

def tic():
    """
    Function to start timer
    Tries to mimic tic() toc() in MATLAB
    """
    StartTime = time.time()
    return StartTime

def toc(StartTime):
    """
    Function to stop timer
    Tries to mimic tic() toc() in MATLAB
    """
    return time.time() - StartTime

def FindLatestModel(CheckPointPath):
    """
    Finds Latest Model in CheckPointPath
    Inputs:
    CheckPointPath - Path where you have stored checkpoints
    Outputs:
    LatestFile - File Name of the latest checkpoint
    """
    FileList = glob.glob(CheckPointPath + '*.ckpt.index') # * means all if need specific format then *.csv
    LatestFile = max(FileList, key=os.path.getctime)
    # Strip everything else except needed information
    LatestFile = LatestFile.replace(CheckPointPath, '')
    LatestFile = LatestFile.replace('.ckpt.index', '')
    return LatestFile


def convertToOneHot(vector, NumClasses):
    """
    Inputs:
    vector - vector of argmax indexes
    NumClasses - Number of classes
    """
    return np.equal.outer(vector, np.arange(NumClasses)).astype(np.float)

### Train your neural network

In [14]:
# # calculating 3 channel mean and std for image dataset
# import tqdm
# import random

# means = np.array([0, 0, 0], dtype=np.float32)
# stds = np.array([0, 0, 0], dtype=np.float32)
# total_images = 0
# randomly_sample = 5000
# for f in tqdm.tqdm(random.sample(glob.glob("dataset_path/**.jpg", recursive = True), randomly_sample)):
#     img = cv2.imread(f)
#     means += img.mean(axis=(0,1))
#     stds += img.std(axis=(0,1))
#     total_images += 1
# means = means / (total_images * 255.)
# stds = stds / (total_images * 255.)
# print("Total images: ", total_images)
# print("Means: ", means)
# print("Stds: ", stds)

In [15]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.optim import AdamW
from torchvision.datasets import CIFAR10
import cv2
import sys
import os
import numpy as np
import random
import skimage
import PIL
import os
import glob
import random
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
import numpy as np
import time
from torchvision.transforms import ToTensor
import argparse
import shutil
import string
from termcolor import colored, cprint
import math as m
from tqdm.notebook import tqdm

def SetupAll(CheckPointPath):
    """
    Inputs: 
    CheckPointPath - Path to save checkpoints/model
    Outputs:
    SaveCheckPoint - Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    ImageSize - Size of the image
    NumTrainSamples - length(Train)
    TrainLabels - Labels corresponding to Train
    NumClasses - Number of classes
    """
    # Read and Setup Labels
    LabelsPathTrain = 'TxtFiles/LabelsTrain.txt'
    TrainLabels = ReadLabels(LabelsPathTrain)

    # If CheckPointPath doesn't exist make the path
    if(not (os.path.isdir(CheckPointPath))):
        os.makedirs(CheckPointPath)
        
    # Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    SaveCheckPoint = 100 
    
    # Image Input Shape
    ImageSize = [32, 32, 3]
    NumTrainSamples = len(TrainSet)

    # Number of classes
    NumClasses = 10

    return SaveCheckPoint, ImageSize, NumTrainSamples, TrainLabels, NumClasses


def ReadLabels(LabelsPathTrain):
    if(not (os.path.isfile(LabelsPathTrain))):
        print('ERROR: Train Labels do not exist in '+LabelsPathTrain)
        sys.exit()
    else:
        TrainLabels = open(LabelsPathTrain, 'r')
        TrainLabels = TrainLabels.read()
        TrainLabels = map(float, TrainLabels.split())

    return TrainLabels
    

def ReadDirNames(ReadPath):
    """
    Inputs: 
    ReadPath is the path of the file you want to read
    Outputs:
    DirNames is the data loaded from TxtFiles/DirNames.txt which has full path to all image files without extension
    """
    # Read text files
    DirNames = open(ReadPath, 'r')
    DirNames = DirNames.read()
    DirNames = DirNames.split()
    return DirNames

    
def GenerateBatch(TrainSet, TrainLabels, ImageSize, MiniBatchSize, Indices):
    """
    Inputs: 
    TrainSet - Variable with Subfolder paths to train files
    NOTE that Train can be replaced by Val/Test for generating batch corresponding to validation (held-out testing in this case)/testing
    TrainLabels - Labels corresponding to Train
    NOTE that TrainLabels can be replaced by Val/TestLabels for generating batch corresponding to validation (held-out testing in this case)/testing
    ImageSize is the Size of the Image
    MiniBatchSize is the size of the MiniBatch
   
    Outputs:
    I1Batch - Batch of images
    LabelBatch - Batch of one-hot encoded labels 
    """
    I1Batch = []
    LabelBatch = []
    
    ImageNum = 0
    while ImageNum < MiniBatchSize:
        # Generate random image
        RandIdx = random.randint(0, len(Indices)-1)
        
        ImageNum += 1

        ##########################################################
        # Add any standardization or data augmentation here!
        ##########################################################

        I1, Label = TrainSet[Indices[RandIdx]]

        # Append All Images and Mask
        I1Batch.append(I1)
        LabelBatch.append(torch.tensor(Label))
        
    return torch.stack(I1Batch), torch.stack(LabelBatch)


def PrettyPrint(NumEpochs, DivTrain, MiniBatchSize, NumTrainSamples, LatestFile):
    """
    Prints all stats with all arguments
    """
    print('Number of Epochs Training will run for ' + str(NumEpochs))
    print('Factor of reduction in training data is ' + str(DivTrain))
    print('Mini Batch Size ' + str(MiniBatchSize))
    print('Number of Training Images ' + str(NumTrainSamples))
    if LatestFile is not None:
        print('Loading latest checkpoint with the name ' + LatestFile)              

def TrainOperation(TrainLabels, NumTrainSamples, ImageSize,
                   NumEpochs, MiniBatchSize, SaveCheckPoint, CheckPointPath,
                   DivTrain, LatestFile, TrainSet, LogsPath, TrainIndices, ValIndices):
    """
    Inputs: 
    TrainLabels - Labels corresponding to Train/Test
    NumTrainSamples - length(Train)
    ImageSize - Size of the image
    NumEpochs - Number of passes through the Train data
    MiniBatchSize is the size of the MiniBatch
    SaveCheckPoint - Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    CheckPointPath - Path to save checkpoints/model
    DivTrain - Divide the data by this number for Epoch calculation, use if you have a lot of dataor for debugging code
    LatestFile - Latest checkpointfile to continue training
    TrainSet - The training dataset
    LogsPath - Path to save Tensorboard Logs
    Outputs:
    Saves Trained network in CheckPointPath and Logs to LogsPath
    """
    # Initialize the model
    model = CIFAR10Model(InputSize=3*32*32,OutputSize=10) 
    ###############################################
    # Fill your optimizer of choice here!
    ###############################################
    Optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.0005)

    # Tensorboard
    # Create a summary to monitor loss tensor
    Writer = SummaryWriter(LogsPath)

    if LatestFile is not None:
        CheckPoint = torch.load(CheckPointPath + LatestFile + '.ckpt')
        # Extract only numbers from the name
        StartEpoch = int(''.join(c for c in LatestFile.split('a')[0] if c.isdigit()))
        model.load_state_dict(CheckPoint['model_state_dict'])
        print('Loaded latest checkpoint with the name ' + LatestFile + '....')
    else:
        StartEpoch = 0
        print('New model initialized....')
        
    history = []
    for Epochs in tqdm(range(StartEpoch, NumEpochs)):
        TrainLosses = []
        
        NumIterationsPerEpoch = int(NumTrainSamples/MiniBatchSize/DivTrain)
        for PerEpochCounter in tqdm(range(NumIterationsPerEpoch)):
            TrainBatch = GenerateBatch(TrainSet, TrainLabels, ImageSize, MiniBatchSize, TrainIndices)
            
            # Predict output with forward pass
            LossThisBatch = model.training_step(TrainBatch)
            TrainLosses.append(LossThisBatch)
            
            Optimizer.zero_grad()
            LossThisBatch.backward()
            Optimizer.step()
            
            # Save checkpoint every some SaveCheckPoint's iterations
            if PerEpochCounter % SaveCheckPoint == 0:
                # Save the Model learnt in this epoch
                SaveName =  CheckPointPath + str(Epochs) + 'a' + str(PerEpochCounter) + 'model.ckpt'
                
                torch.save({'epoch': Epochs,'model_state_dict': model.state_dict(),'optimizer_state_dict': Optimizer.state_dict(),'loss': LossThisBatch}, SaveName)
                print('\n' + SaveName + ' Model Saved...')

            result = model.validation_step(TrainBatch)
            result['TrainLoss'] = torch.stack(TrainLosses).mean().item()
            model.epoch_end(Epochs*NumIterationsPerEpoch + PerEpochCounter, result)
            history.append(result)
            
            # Tensorboard
            Writer.add_scalar('LossEveryIter', result["loss"], Epochs*NumIterationsPerEpoch + PerEpochCounter)
            Writer.add_scalar('Accuracy', result["acc"], Epochs*NumIterationsPerEpoch + PerEpochCounter)
            # If you don't flush the tensorboard doesn't update until a lot of iterations!
            Writer.flush()

        ValBatch = GenerateBatch(TrainSet, TrainLabels, ImageSize, MiniBatchSize, ValIndices)
        ValResult = model.validation_step(ValBatch)
        Writer.add_scalar('ValidationLossEveryEpoch',ValResult['loss'],Epochs)
        Writer.add_scalar('ValidationAccuracyEveryEpoch',ValResult['acc'],Epochs)
        Writer.flush()
        
        # Save model every epoch
        SaveName = CheckPointPath + str(Epochs) + 'model.ckpt'
        torch.save({'epoch': Epochs,'model_state_dict': model.state_dict(),'optimizer_state_dict': Optimizer.state_dict(),'loss': LossThisBatch}, SaveName)
        print('\n' + SaveName + ' Model Saved...')
        


# Default Hyperparameters
NumEpochs = 20

normalize = torchvision.transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                             std=[0.2470, 0.2435, 0.2616])

# transforms_to_apply = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor(),normalize])
transformations = transforms.Compose([transforms.ToTensor(), normalize])
TrainSet = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transformations)
ValSet = torchvision.datasets.CIFAR10(root='../../data', train=True,
                                        download=True, transform=transformations)

valid_size = 0.1
num_train = len(TrainSet)
indices = list (range(num_train))
split = int(np.floor(valid_size * num_train))

np.random.seed(42)
np.random.shuffle(indices)

TrainIndices, ValIndices = indices[split:], indices[:split]


DivTrain = 1.0
MiniBatchSize = 1024
LoadCheckPoint = 0
CheckPointPath = "Checkpoints/"
LogsPath = "Logs"

# Setup all needed paraymeters including file reading
SaveCheckPoint, ImageSize, NumTrainSamples, TrainLabels, NumClasses = SetupAll(CheckPointPath)

# Find Latest Checkpoint File
if LoadCheckPoint==1:
    LatestFile = FindLatestModel(CheckPointPath)
else:
    LatestFile = None

# Pretty print stats
PrettyPrint(NumEpochs, DivTrain, MiniBatchSize, NumTrainSamples, LatestFile)

TrainOperation(TrainLabels, NumTrainSamples, ImageSize,
                NumEpochs, MiniBatchSize, SaveCheckPoint, CheckPointPath,
                DivTrain, LatestFile, TrainSet, LogsPath, TrainIndices, ValIndices)

Files already downloaded and verified
Files already downloaded and verified
Number of Epochs Training will run for 20
Factor of reduction in training data is 1.0
Mini Batch Size 1024
Number of Training Images 50000
New model initialized....


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]


Checkpoints/0a0model.ckpt Model Saved...
Epoch [0], loss: 2.2600, acc: 0.1729
Epoch [1], loss: 2.1968, acc: 0.2002
Epoch [2], loss: 2.1570, acc: 0.1973
Epoch [3], loss: 2.0991, acc: 0.2607
Epoch [4], loss: 2.0485, acc: 0.2490
Epoch [5], loss: 1.9643, acc: 0.3037
Epoch [6], loss: 1.9474, acc: 0.3057
Epoch [7], loss: 1.9341, acc: 0.3066
Epoch [8], loss: 1.8889, acc: 0.3008
Epoch [9], loss: 1.8796, acc: 0.3301
Epoch [10], loss: 1.8339, acc: 0.3623
Epoch [11], loss: 1.7888, acc: 0.3730
Epoch [12], loss: 1.8112, acc: 0.3252
Epoch [13], loss: 1.7889, acc: 0.3730
Epoch [14], loss: 1.7660, acc: 0.3691
Epoch [15], loss: 1.8019, acc: 0.3740
Epoch [16], loss: 1.7699, acc: 0.3770
Epoch [17], loss: 1.7478, acc: 0.3564
Epoch [18], loss: 1.7133, acc: 0.4004
Epoch [19], loss: 1.7011, acc: 0.3994
Epoch [20], loss: 1.7475, acc: 0.3867
Epoch [21], loss: 1.7240, acc: 0.3838
Epoch [22], loss: 1.6905, acc: 0.3955
Epoch [23], loss: 1.7303, acc: 0.3867
Epoch [24], loss: 1.6520, acc: 0.4082
Epoch [25], loss: 

  0%|          | 0/48 [00:00<?, ?it/s]

...


Checkpoints/17a0model.ckpt Model Saved...
Epoch [816], loss: 0.4759, acc: 0.8418
Epoch [817], loss: 0.4486, acc: 0.8584
Epoch [818], loss: 0.5138, acc: 0.8281
Epoch [819], loss: 0.4650, acc: 0.8486
Epoch [820], loss: 0.5045, acc: 0.8203
Epoch [821], loss: 0.5015, acc: 0.8242
Epoch [822], loss: 0.5561, acc: 0.8096
Epoch [823], loss: 0.5094, acc: 0.8135
Epoch [824], loss: 0.4958, acc: 0.8369
Epoch [825], loss: 0.4971, acc: 0.8369
Epoch [826], loss: 0.4751, acc: 0.8535
Epoch [827], loss: 0.5272, acc: 0.8252
Epoch [828], loss: 0.5047, acc: 0.8262
Epoch [829], loss: 0.5295, acc: 0.8096
Epoch [830], loss: 0.4830, acc: 0.8359
Epoch [831], loss: 0.4808, acc: 0.8379
Epoch [832], loss: 0.4906, acc: 0.8408
Epoch [833], loss: 0.4931, acc: 0.8350
Epoch [834], loss: 0.5033, acc: 0.8213
Epoch [835], loss: 0.4822, acc: 0.8447
Epoch [836], loss: 0.4914, acc: 0.8398
Epoch [837], loss: 0.5426, acc: 0.8174
Epoch [838], loss: 0.5204, acc: 0.8320
Epoch [839], loss: 0.4878, acc: 0.8271
Epoch [840], loss: 0.

  0%|          | 0/48 [00:00<?, ?it/s]


Checkpoints/18a0model.ckpt Model Saved...
Epoch [864], loss: 0.4232, acc: 0.8613
Epoch [865], loss: 0.4456, acc: 0.8555
Epoch [866], loss: 0.4406, acc: 0.8506
Epoch [867], loss: 0.4869, acc: 0.8408
Epoch [868], loss: 0.4531, acc: 0.8457
Epoch [869], loss: 0.4653, acc: 0.8408
Epoch [870], loss: 0.4408, acc: 0.8486
Epoch [871], loss: 0.4256, acc: 0.8613
Epoch [872], loss: 0.4486, acc: 0.8613
Epoch [873], loss: 0.4440, acc: 0.8574
Epoch [874], loss: 0.4339, acc: 0.8643
Epoch [875], loss: 0.4430, acc: 0.8604
Epoch [876], loss: 0.4226, acc: 0.8525
Epoch [877], loss: 0.3956, acc: 0.8779
Epoch [878], loss: 0.3689, acc: 0.8867
Epoch [879], loss: 0.4469, acc: 0.8506
Epoch [880], loss: 0.4201, acc: 0.8633
Epoch [881], loss: 0.4242, acc: 0.8613
Epoch [882], loss: 0.4156, acc: 0.8555
Epoch [883], loss: 0.4555, acc: 0.8457
Epoch [884], loss: 0.4540, acc: 0.8584
Epoch [885], loss: 0.3883, acc: 0.8809
Epoch [886], loss: 0.4472, acc: 0.8506
Epoch [887], loss: 0.4525, acc: 0.8477
Epoch [888], loss: 0.

  0%|          | 0/48 [00:00<?, ?it/s]


Checkpoints/19a0model.ckpt Model Saved...
Epoch [912], loss: 0.4624, acc: 0.8516
Epoch [913], loss: 0.3579, acc: 0.8945
Epoch [914], loss: 0.4571, acc: 0.8477
Epoch [915], loss: 0.4673, acc: 0.8340
Epoch [916], loss: 0.4170, acc: 0.8467
Epoch [917], loss: 0.4312, acc: 0.8604
Epoch [918], loss: 0.4114, acc: 0.8711
Epoch [919], loss: 0.4043, acc: 0.8682
Epoch [920], loss: 0.4316, acc: 0.8496
Epoch [921], loss: 0.4178, acc: 0.8730
Epoch [922], loss: 0.4473, acc: 0.8506
Epoch [923], loss: 0.4515, acc: 0.8457
Epoch [924], loss: 0.4072, acc: 0.8662
Epoch [925], loss: 0.4212, acc: 0.8701
Epoch [926], loss: 0.4120, acc: 0.8721
Epoch [927], loss: 0.4088, acc: 0.8623
Epoch [928], loss: 0.4255, acc: 0.8525
Epoch [929], loss: 0.4081, acc: 0.8682
Epoch [930], loss: 0.4086, acc: 0.8740
Epoch [931], loss: 0.4111, acc: 0.8691
Epoch [932], loss: 0.4320, acc: 0.8574
Epoch [933], loss: 0.4281, acc: 0.8633
Epoch [934], loss: 0.4285, acc: 0.8584
Epoch [935], loss: 0.4138, acc: 0.8711
Epoch [936], loss: 0.

### Test your neural network

In [17]:
import cv2
import os
import sys
import glob
import random
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
import numpy as np
import time
from torchvision.transforms import ToTensor
import argparse
import shutil
import string
import math as m
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
import torch

def SetupAll():
    """
    Outputs:
    ImageSize - Size of the Image
    """   
    # Image Input Shape
    ImageSize = [32, 32, 3]

    return ImageSize

def StandardizeInputs(Img):
    ##########################################################################
    # Add any standardization or cropping/resizing if used in Training here!
    ##########################################################################
    return Img
    
def ReadImages(Img):
    """
    Outputs:
    I1Combined - I1 image after any standardization and/or cropping/resizing to ImageSize
    I1 - Original I1 image for visualization purposes only
    """    
    I1 = Img
    
    if(I1 is None):
        # OpenCV returns empty list if image is not read! 
        print('ERROR: Image I1 cannot be read')
        sys.exit()
        
    I1S = StandardizeInputs(np.float32(I1))

    I1Combined = np.expand_dims(I1S, axis=0)

    return I1Combined, I1
                

def TestOperation(ImageSize, ModelPath, TestSet, LabelsPathPred):
    """
    Inputs: 
    ImageSize is the size of the image
    ModelPath - Path to load trained model from
    TestSet - The test dataset
    LabelsPathPred - Path to save predictions
    Outputs:
    Predictions written to TxtFiles/PredOut.txt
    """
    # Predict output with forward pass, MiniBatchSize for Test is 1
    model = CIFAR10Model(InputSize=3*32*32,OutputSize=10) 
    
    CheckPoint = torch.load(ModelPath)
    model.load_state_dict(CheckPoint['model_state_dict'])
    print('Number of parameters in this model are %d ' % len(model.state_dict().items()))
    
    OutSaveT = open(LabelsPathPred, 'w')

    for count in tqdm(range(len(TestSet))): 
        Img, Label = TestSet[count]
        Img, ImgOrg = ReadImages(Img)
        PredT = torch.argmax(model(torch.tensor(Img))).item()

        OutSaveT.write(str(PredT)+'\n')
    OutSaveT.close()

def Accuracy(Pred, GT):
    """
    Inputs: 
    Pred are the predicted labels
    GT are the ground truth labels
    Outputs:
    Accuracy in percentage
    """
    return (np.sum(np.array(Pred)==np.array(GT))*100.0/len(Pred))

def ReadLabels(LabelsPathTest, LabelsPathPred):
    if(not (os.path.isfile(LabelsPathTest))):
        print('ERROR: Test Labels do not exist in '+LabelsPathTest)
        sys.exit()
    else:
        LabelTest = open(LabelsPathTest, 'r')
        LabelTest = LabelTest.read()
        LabelTest = map(float, LabelTest.split())

    if(not (os.path.isfile(LabelsPathPred))):
        print('ERROR: Pred Labels do not exist in '+LabelsPathPred)
        sys.exit()
    else:
        LabelPred = open(LabelsPathPred, 'r')
        LabelPred = LabelPred.read()
        LabelPred = map(float, LabelPred.split())
        
    return LabelTest, LabelPred

def ConfusionMatrix(LabelsTrue, LabelsPred):
    """
    LabelsTrue - True labels
    LabelsPred - Predicted labels
    """

    # Get the confusion matrix using sklearn.
    LabelsTrue, LabelsPred = list(LabelsTrue), list(LabelsPred)
    cm = confusion_matrix(y_true=LabelsTrue,  # True class for test-set.
                          y_pred=LabelsPred)  # Predicted class.

    # Print the confusion matrix as text.
    for i in range(10):
        print(str(cm[i, :]) + ' ({0})'.format(i))

    # Print the class-numbers for easy reference.
    class_numbers = [" ({0})".format(i) for i in range(10)]
    print("".join(class_numbers))

    print('Accuracy: '+ str(Accuracy(LabelsPred, LabelsTrue)), '%')


ModelPath = "Checkpoints/19model.ckpt"
LabelsPath = "TxtFiles/LabelsTest.txt"

TestSet = CIFAR10(root='data/', train=False, transform=transformations)

# Setup all needed parameters including file reading
ImageSize = SetupAll()

# Define PlaceHolder variables for Input and Predicted output
LabelsPathPred = 'TxtFiles/PredOut.txt' # Path to save predicted labels

TestOperation(ImageSize, ModelPath, TestSet, LabelsPathPred)

# Plot Confusion Matrix
LabelsTrue, LabelsPred = ReadLabels(LabelsPath, LabelsPathPred)
ConfusionMatrix(LabelsTrue, LabelsPred) 

Number of parameters in this model are 10 


  0%|          | 0/10000 [00:00<?, ?it/s]

[624  38  49  17  42  15  15  27  97  76] (0)
[ 44 648  18  24  10   7  14  19  61 155] (1)
[ 76  16 435  74 133  89  63  75  15  24] (2)
[ 23  19  80 336  65 232  91  89  27  38] (3)
[ 46  16 122  60 473  55  79 101  28  20] (4)
[ 16  14  78 161  71 488  44  86  17  25] (5)
[ 13  16  78  90  94  68 577  21  12  31] (6)
[ 31   6  29  53  71  97  14 655   9  35] (7)
[106  70  17  21  20  18  12  12 660  64] (8)
[ 49 159  15  36  16  22  17  46  34 606] (9)
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)
Accuracy: 55.02 %
